In [0]:
# -*- coding: utf-8 -*-
import dataiku
import datetime
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from dataiku import pandasutils as pdu

In [0]:
# Connect to folder
sample_batches = dataiku.Folder("zY3ZURve")
sample_batches_info = sample_batches.get_info()

# Path in folder
data_folder = dataiku.get_custom_variables()["batches_path"]

In [0]:
# Model
seed = 112233
np.random.seed(seed)
reference_time = datetime.datetime(2024, 1, 1)

def get_features(current_time: datetime.datetime, size=(1000,1)) -> pd.DataFrame:
    time_delta = current_time - reference_time
    # Trend increses in time to saturation
    trend = min((0.008 * time_delta.total_seconds() / (24 * 60 * 60)), 1) * np.ones(size)
    # Random noise
    noise_a = np.random.normal(0, 0.1, size)
    
#     df = pd.DataFrame(
#         {
#             "unixtime": times,
#             "feature_a": 
#         }
#     )
    
    return df

def get_outcome(features: pd.DataFrame) -> pd.Series:
    return 

In [0]:
# Produce the data and save to the folder
current_time = datetime.datetime.now()
print(current_time)
time_delta = (current_time - reference_time)
print(time_delta.days)
size = (1000, 1)
print(min((0.008 * time_delta.total_seconds() / (24 * 60 * 60)), 1) * np.ones(size))


In [0]:
# Produce features and the 
features = get_features(current_time)
features['outcome'] = get_outcome(features)